In [1]:
import csv
import numpy as np
import pandas as pd
from pyspark import SparkContext
from cassandra.cluster import Cluster

### Reading dataset and pre-processing

In [597]:
def preprocessing(input_path, updated_data_path):
    all_epi_indicator_df = pd.read_csv(input_path, sep=",")
    all_epi_indicator_df = all_epi_indicator_df.fillna(0)
#     print(all_epi_indicator_df.head())
    all_epi_indicator_df.to_csv(output_path)
    return all_epi_indicator_df

### Getting column average dataframe to save to cassandra

In [418]:
def get_column_avg_df(updated_data_path, column_list, index_list, state_list):
    # Create RDD
    rdd = sparkContext.textFile(updated_data_path)
    rdd = rdd.mapPartitions(lambda x: csv.reader(x))
    header = rdd.first()
    rdd = rdd.filter(lambda x: x != header)

    epi_avg_dict = dict()
    column_dict = dict()
    
    # Iterate over each column
    for i in range(0, len(index_list)):
        # Get 
        temp_rdd = rdd.map(lambda x: (x[1], float(x[index_list[i] + 1])))
        temp_rdd = temp_rdd.mapValues(lambda row:(row,1))
        temp_rdd = temp_rdd.reduceByKey(lambda x1, x2: (x1[0] + x2[0], x1[1] + x2[1]))
        avg_by_key = temp_rdd.mapValues(lambda val: round((val[0] / val[1]), 4)).collectAsMap()
        epi_avg_dict[column_list[i]] =  avg_by_key
    
    # Create a dictionary of columns to create a dataframe
    column_dict['state_abbv'] = list(state_list)
    for key in epi_avg_dict.keys():
        column_dict[key] = list(epi_avg_dict[key].values())

    result_df = pd.DataFrame(column_dict)
    
    return result_df

In [566]:
def get_column_df(updated_data_path, column_list):
    input_df = pd.read_csv(updated_data_path)
    result_dict = dict()
    for item in column_list:
        result_dict[item] = list(input_df[item])
    output_df = pd.DataFrame(result_dict) 
    return output_df

## Storing data to Cassandra

In [ ]:
## Connecting to cassandra cluster 

In [355]:
def connect_to_cassandra_cluster(cluster_ip):
    cluster = Cluster([cluster_ip])  # provide contact points and port
    session = cluster.connect()
    return session

In [356]:
## Create or use existing keyspace

In [357]:
def create_or_connect_keyspace(keyspace, session):
    query = "CREATE KEYSPACE IF NOT EXISTS " + keyspace + " WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 }"
    session.execute(query)
    session.set_keyspace(keyspace)

In [358]:
## Create a table for non_voter_avg of each state

In [373]:
def create_epi_avg_table(session, keyspace, table_name, column_list):
    session.set_keyspace(keyspace)
    query = "CREATE TABLE " + table_name + " (state_abbv text PRIMARY KEY, " + ", ".join(column_name + " float" for column_name in column_list) + ")"
    session.execute(query)

In [ ]:
## create normal table

In [570]:
# def create_epi_table(session, keyspace, table_name, column_list):
#     session.set_keyspace(keyspace)
#     query = "CREATE TABLE " + table_name + " (state_abbv text, " + ", ".join(column_name + " float" if column_name != 'state_abbv' else column_name + " text" for column_name in column_list) + ")"
#     session.execute(query)

In [374]:
## Populate cassandra table

In [415]:
def populate_epi_table(session, keyspace, table_name, column_list, epi_avg_df):
    session.set_keyspace(keyspace)
    query = "INSERT INTO " + table_name + " (state_abbv, " + ", ".join(column_name for column_name in column_list) + ") VALUES ( " + ", ".join('?' for i in range(0, len(column_list)+1)) + ")"
    prepared = session.prepare(query)
    for index, row in epi_avg_df.iterrows():
        parameter_list = []
        parameter_list.append(row.state_abbv)
        for column in column_list:
            parameter_list.append(row[column])
        session.execute(prepared, tuple(parameter_list))

In [354]:
def get_table_data(table_name):
    column_dict = dict()
    state_names = []
    epi_avgs = []
    result = session.execute("Select * from " + table_name)
    column_dict['state_abbv'] = []
    for row in result:
        
        column_dict['state_abbv'].append(row.state_abbv)
        column_dict[].append(row.state_abbv)
        
        
        epi_avgs.append(row.index_2016)
    result_df = pd.DataFrame({"States": state_names, "EPI Average": epi_avgs})
    return result_df

### Declare initial variables

In [401]:
input_path = "./Dataset/epi_indicators-all_years.csv"
updated_data_path = "./Dataset/updated_epi_indicators.csv"
cluster_ip = '54.164.78.8'
keyspace = 'election'

### Create DF of all EPIs

In [345]:
all_epi_indicator_df = preprocessing(input_path, updated_data_path)

In [326]:
state_list = all_epi_indicator_df['state_abbv'].unique()

In [398]:
sparkContext = SparkContext.getOrCreate()

In [370]:
session = connect_to_cassandra_cluster(cluster_ip)

In [371]:
create_or_connect_keyspace(keyspace, session)

In [538]:
table_name = 'Table6'
column_list = ['year', 'website_precinct_ballot', 'prov_rej_all']
index_list = [list(all_epi_indicator_df.columns).index(column) for column in column_list]

In [539]:
result_df = get_column_avg_df(updated_data_path, column_list, index_list, state_list)

In [542]:
result_df.head()

,state_abbv,year,website_precinct_ballot,prov_rej_all
0,AK,2012.0,0.2,0.0029
1,AL,2012.0,0.6,0.0013
2,AR,2012.0,0.0,0.0097
3,AZ,2012.0,0.0,0.0020
4,CA,2012.0,0.2,0.0003


In [534]:
### Cassandra operations

In [535]:
create_epi_avg_table(session, keyspace, table_name, column_list)

In [536]:
populate_epi_table(session, keyspace, table_name, column_list, result_df)

In [537]:
## Table 5 post processing for result_df

In [ ]:
result_df['sum_epi'] = [result_df['nonvoter_illness_pct'][i] + result_df['nonvoter_reg_pct'][i] for i in range(0, result_df.shape[0])]
result_df = result_df.drop(columns=['nonvoter_illness_pct', 'nonvoter_reg_pct'])
column_list = ['vep_turnout', 'sum_epi']

In [ ]:
## Table 2 

In [567]:
table_name = 'Table2'
column_list = ['state_abbv', 'reg_rej', 'online_reg', 'vep_turnout', 'year']
result_df = get_column_df(updated_data_path, column_list)

In [581]:
# result_df

In [576]:
session.set_keyspace(keyspace)
query = "CREATE TABLE Table2 (key float primary key, state_abbv text, reg_rej float, online_reg float, vep_turnout float, year int)"
session.execute(query)

In [580]:
session.set_keyspace(keyspace)
query = "INSERT INTO Table2 (key, state_abbv, reg_rej, online_reg, vep_turnout, year) values (?, ?, ?, ?, ?, ?)"
prepared = session.prepare(query)
for index, row in result_df.iterrows():
    parameter_list = []
    parameter_list.append(index)
    parameter_list.append(row['state_abbv'])
    parameter_list.append(row['reg_rej'])
    parameter_list.append(row['online_reg'])
    parameter_list.append(row['vep_turnout'])
    parameter_list.append(row['year'])
    session.execute(prepared, tuple(parameter_list))

In [ ]:
## Table 6 

In [582]:
table_name = 'Table6'
column_list = ['state_abbv', 'year', 'website_precinct_ballot', 'prov_rej_all']
result_df = get_column_df(updated_data_path, column_list)

In [583]:
result_df.head()

,state_abbv,year,website_precinct_ballot,prov_rej_all
0,AK,2008,0.0,0.000781
1,AK,2010,0.0,0.004978
2,AK,2012,0.0,0.000810
3,AK,2014,0.0,0.007206
4,AK,2016,1.0,0.000838


In [584]:
session.set_keyspace(keyspace)
query = "CREATE TABLE Table6 (key float primary key, state_abbv text, year int, website_precinct_ballot float, prov_rej_all float)"
session.execute(query)

In [585]:
session.set_keyspace(keyspace)
query = "INSERT INTO Table6 (key, state_abbv, year, website_precinct_ballot, prov_rej_all) values (?, ?, ?, ?, ?)"
prepared = session.prepare(query)
for index, row in result_df.iterrows():
    parameter_list = []
    parameter_list.append(index)
    parameter_list.append(row['state_abbv'])
    parameter_list.append(row['year'])
    parameter_list.append(row['website_precinct_ballot'])
    parameter_list.append(row['prov_rej_all'])
    session.execute(prepared, tuple(parameter_list))

In [586]:
## Table7

In [783]:
table_name = 'Table7'
column_list_1 = ['state_abbv', 'vep_turnout', 'year']
column_list_2 = ['State Fips', 'GDP', 'year']

input_path_1 = "./Dataset/epi_indicators-all_years.csv"
updated_data_path_1 = "./Dataset/updated_epi_indicators-all_years.csv"

input_path_2 = "./Dataset/state_vs_gdp.csv"
updated_data_path_2 = "./Dataset/updated_state_vs_gdp.csv"

input_df_1 = pd.read_csv(input_path_1, sep=",")
input_df_1 = input_df_1.fillna(0)
input_df_1.to_csv(updated_data_path_1)

input_df_2 = pd.read_csv(input_path_2, sep=",")
input_df_2 = input_df_2.fillna(0)
input_df_2.to_csv(updated_data_path_2)

# input_df_1 = preprocessing(, )
# input_df_2 = preprocessing(, )

filterd_df_1 = get_column_df(updated_data_path_1, column_list_1)
filterd_df_2 = get_column_df(updated_data_path_2, column_list_2)

In [784]:
name_abbv_dic = dict()

In [785]:
# filterd_df_1.head()

In [786]:
# filterd_df_1['year'].unique()

In [787]:
# filterd_df_2.head()

In [788]:
# filterd_df_2['year'].unique()

In [789]:
# len(filterd_df_2['State Fips'].unique())

In [790]:
state_abbv_df = pd.read_csv('./Dataset/name_abbv_file.csv', header=None)
state_abbv_df.columns=['State Fip', 'state_abbv']
state_abbv_df = state_abbv_df.dropna(how='all')

In [791]:
state_abbv_df.head()

,State Fip,state_abbv
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [792]:
name_abbv_dic = {state_abbv_df['State Fip'][i]: state_abbv_df['state_abbv'][i] for i in range(state_abbv_df.shape[0])} 
abbv_name_dic = {state_abbv_df['state_abbv'][i]: state_abbv_df['State Fip'][i] for i in range(state_abbv_df.shape[0])} 

In [793]:
# abbv_name_dic

In [797]:
state_name = []
state_abbv = []
gdp = []
year = []

for index, row in filterd_df_2.iterrows():
    if(row['State Fips'] in list(state_abbv_df['State Fip'])):
        state_name.append(row['State Fips'])
        state_abbv.append(name_abbv_dic[row['State Fips']])
        gdp.append(row['GDP'])
        year.append(row['year'])

result_df = pd.DataFrame({"State Fips":state_name, "state_abbv":state_abbv, "GDP":gdp, "year":year})

In [798]:
result_df.head(-10)

,State Fips,state_abbv,GDP,year
0,Alabama,AL,186849.0,2014
1,Alaska,AK,53273.0,2014
2,Arizona,AZ,273677.1,2014
3,Arkansas,AR,112932.1,2014
4,California,CA,2312540.1,2014
...,...,...,...,...
138,Oklahoma,OK,173483.7,2012
139,Oregon,OR,174493.1,2012
140,Pennsylvania,PA,641317.3,2012
141,Rhode Island,RI,51642.0,2012


In [799]:
state_year_turnout_dict = dict()
year_turnout_dict = dict()
for name, group_1 in filterd_df_1.groupby('state_abbv'):
    for year, group_2 in group_1.groupby('year'):
        for index, row in group_2.iterrows():
            state_year_turnout_dict[str(name)+","+str(year)] = row.vep_turnout

In [800]:
state_year_turnout_dict

{'AK,2008': 0.68277264,
 'AK,2010': 0.52556038,
 'AK,2012': 0.59180635,
 'AK,2014': 0.54381794,
 'AK,2016': 0.61800861,
 'AL,2008': 0.60952842,
 'AL,2010': 0.43060836,
 'AL,2012': 0.58890975,
 'AL,2014': 0.33212399,
 'AL,2016': 0.59257072,
 'AR,2008': 0.52904886,
 'AR,2010': 0.37541923,
 'AR,2012': 0.50954992,
 'AR,2014': 0.40142861,
 'AR,2016': 0.53103119,
 'AZ,2008': 0.57360554,
 'AZ,2010': 0.41521883,
 'AZ,2012': 0.53292167,
 'AZ,2014': 0.34115994,
 'AZ,2016': 0.56217176,
 'CA,2008': 0.61735171,
 'CA,2010': 0.45050842,
 'CA,2012': 0.55893815,
 'CA,2014': 0.30784670000000003,
 'CA,2016': 0.5840137,
 'CO,2008': 0.71601105,
 'CO,2010': 0.50696188,
 'CO,2012': 0.71049291,
 'CO,2014': 0.54463571,
 'CO,2016': 0.72087783,
 'CT,2008': 0.66642547,
 'CT,2010': 0.45959899,
 'CT,2012': 0.60998958,
 'CT,2014': 0.42533553,
 'CT,2016': 0.65427256,
 'DC,2008': 0.6169486,
 'DC,2010': 0.29573914,
 'DC,2012': 0.61965692,
 'DC,2014': 0.35791108,
 'DC,2016': 0.61113918,
 'DE,2008': 0.65832853,
 'DE,2010

In [801]:
vep_turnout_list = []
for index, row in result_df.iterrows():
    vep_turnout_list.append(state_year_turnout_dict[str(row.state_abbv) + "," + str(row.year)])

In [802]:
print(len(result_df))
print(len(vep_turnout_list))

153
153


In [803]:
result_df['vep_turnout'] = vep_turnout_list

In [804]:
result_df.head()

,State Fips,state_abbv,GDP,year,vep_turnout
0,Alabama,AL,186849.0,2014,0.332124
1,Alaska,AK,53273.0,2014,0.543818
2,Arizona,AZ,273677.1,2014,0.341160
3,Arkansas,AR,112932.1,2014,0.401429
4,California,CA,2312540.1,2014,0.307847


In [805]:
result_df = result_df.drop(columns=['state_abbv'])

In [806]:
result_df.head()

,State Fips,GDP,year,vep_turnout
0,Alabama,186849.0,2014,0.332124
1,Alaska,53273.0,2014,0.543818
2,Arizona,273677.1,2014,0.341160
3,Arkansas,112932.1,2014,0.401429
4,California,2312540.1,2014,0.307847


In [808]:
session.set_keyspace(keyspace)
query = "CREATE TABLE Table7 (key float primary key, state_fips text, year int, gdp float, vep_turnout float)"
session.execute(query)

In [809]:
session.set_keyspace(keyspace)
query = "INSERT INTO Table7 (key, state_fips, year, gdp, vep_turnout) values (?, ?, ?, ?, ?)"
prepared = session.prepare(query)
for index, row in result_df.iterrows():
    parameter_list = []
    parameter_list.append(index)
    parameter_list.append(row['State Fips'])
    parameter_list.append(row['year'])
    parameter_list.append(row['GDP'])
    parameter_list.append(row['vep_turnout'])
    session.execute(prepared, tuple(parameter_list))

In [ ]:
## Table8 

In [810]:
input_path = "./Dataset/epi_indicators-all_years.csv"
updated_data_path = "./Dataset/updated_epi_indicators.csv"
cluster_ip = '54.164.78.8'
keyspace = 'election'

table_name = 'Table8'
column_list = ['state_abbv', 'vep_turnout', 'wait', 'year']

result_dict = dict()

preprocessing(input_path, updated_data_path)

result_df = get_column_df(updated_data_path, column_list)

In [811]:
result_df[:10]

,state_abbv,vep_turnout,wait,year
0,AK,0.682773,5.712700,2008
1,AK,0.525560,0.000000,2010
2,AK,0.591806,3.656872,2012
3,AK,0.543818,4.156598,2014
4,AK,0.618009,6.428843,2016
5,AL,0.609528,14.260803,2008
6,AL,0.430608,0.000000,2010
7,AL,0.588910,11.308345,2012
8,AL,0.332124,3.716310,2014
9,AL,0.592571,13.146628,2016


In [812]:
state_name = []
for index, row in result_df.iterrows():
    state_name.append(abbv_name_dic[row['state_abbv']])

result_df['state_fips'] = state_name

In [813]:
result_df

,state_abbv,vep_turnout,wait,year,state_fips
0,AK,0.682773,5.712700,2008,Alaska
1,AK,0.525560,0.000000,2010,Alaska
2,AK,0.591806,3.656872,2012,Alaska
3,AK,0.543818,4.156598,2014,Alaska
4,AK,0.618009,6.428843,2016,Alaska
...,...,...,...,...,...
250,WY,0.631045,5.620379,2008,Wyoming
251,WY,0.458080,0.000000,2010,Wyoming
252,WY,0.592977,4.516539,2012,Wyoming
253,WY,0.393362,2.759438,2014,Wyoming


In [815]:
result_df = result_df.drop(columns=['state_abbv'])

In [816]:
result_df.head()

,vep_turnout,wait,year,state_fips
0,0.682773,5.712700,2008,Alaska
1,0.525560,0.000000,2010,Alaska
2,0.591806,3.656872,2012,Alaska
3,0.543818,4.156598,2014,Alaska
4,0.618009,6.428843,2016,Alaska


In [817]:
session.set_keyspace(keyspace)
query = "CREATE TABLE Table8 (key float primary key, state_fips text, year int, wait float, vep_turnout float)"
session.execute(query)

In [818]:
session.set_keyspace(keyspace)
query = "INSERT INTO Table8 (key, state_fips, year, wait, vep_turnout) values (?, ?, ?, ?, ?)"
prepared = session.prepare(query)
for index, row in result_df.iterrows():
    parameter_list = []
    parameter_list.append(index)
    parameter_list.append(row['state_fips'])
    parameter_list.append(row['year'])
    parameter_list.append(row['wait'])
    parameter_list.append(row['vep_turnout'])
    session.execute(prepared, tuple(parameter_list))

In [819]:
## Table9

In [820]:
table_name = 'Table9'
column_list_1 = ['state_abbv', 'vep_turnout', 'year']
column_list_2 = ['State Fips', 'Age group', 'Year', 'Data']

input_path_1 = "./Dataset/epi_indicators-all_years.csv"
updated_data_path_1 = "./Dataset/updated_epi_indicators-all_years.csv"

input_path_2 = "./Dataset/state_vs_age.csv"
updated_data_path_2 = "./Dataset/updated_state_vs_age.csv"

input_df_1 = pd.read_csv(input_path_1, sep=",")
input_df_1 = input_df_1.fillna(0)
input_df_1.to_csv(updated_data_path_1)

input_df_2 = pd.read_csv(input_path_2, sep=",")
input_df_2 = input_df_2.fillna(0)
input_df_2.to_csv(updated_data_path_2)

# input_df_1 = preprocessing(, )
# input_df_2 = preprocessing(, )

filterd_df_1 = get_column_df(updated_data_path_1, column_list_1)
filterd_df_2 = get_column_df(updated_data_path_2, column_list_2)

In [822]:
filterd_df_1.head()

,state_abbv,vep_turnout,year
0,AK,0.682773,2008
1,AK,0.525560,2010
2,AK,0.591806,2012
3,AK,0.543818,2014
4,AK,0.618009,2016


In [823]:
filterd_df_2.head()

,State Fips,Age group,Year,Data
0,Alabama,Ages 65 and over,2000,580880
1,Alabama,Ages 65 and over,2001,583831
2,Alabama,Ages 65 and over,2002,585605
3,Alabama,Ages 65 and over,2003,590372
4,Alabama,Ages 65 and over,2004,593826


In [833]:
state_fips = []
state_abbv = []
age_group = []
year = []
data = []

for index, row in filterd_df_2.iterrows():
    if(row['State Fips'] in list(state_abbv_df['State Fip']) and row['Year'] in list(filterd_df_1['year'])):
        state_fips.append(row['State Fips'])
        state_abbv.append(name_abbv_dic[row['State Fips']])
        age_group.append(row['Age group'])
        year.append(row['Year'])
        data.append(row['Data'])

result_df = pd.DataFrame({"state_fips":state_fips, "state_abbv":state_abbv, "age_group":age_group, "year":year, "data":data})

In [834]:
result_df.head()

,state_fips,state_abbv,age_group,year,data
0,Alabama,AL,Ages 65 and over,2008,636944
1,Alabama,AL,Ages 65 and over,2010,660854
2,Alabama,AL,Ages 65 and over,2012,698055
3,Alabama,AL,Ages 65 and over,2014,741948
4,Alabama,AL,Ages 65 and over,2016,783936


In [835]:
state_year_turnout_dict

{'AK,2008': 0.68277264,
 'AK,2010': 0.52556038,
 'AK,2012': 0.59180635,
 'AK,2014': 0.54381794,
 'AK,2016': 0.61800861,
 'AL,2008': 0.60952842,
 'AL,2010': 0.43060836,
 'AL,2012': 0.58890975,
 'AL,2014': 0.33212399,
 'AL,2016': 0.59257072,
 'AR,2008': 0.52904886,
 'AR,2010': 0.37541923,
 'AR,2012': 0.50954992,
 'AR,2014': 0.40142861,
 'AR,2016': 0.53103119,
 'AZ,2008': 0.57360554,
 'AZ,2010': 0.41521883,
 'AZ,2012': 0.53292167,
 'AZ,2014': 0.34115994,
 'AZ,2016': 0.56217176,
 'CA,2008': 0.61735171,
 'CA,2010': 0.45050842,
 'CA,2012': 0.55893815,
 'CA,2014': 0.30784670000000003,
 'CA,2016': 0.5840137,
 'CO,2008': 0.71601105,
 'CO,2010': 0.50696188,
 'CO,2012': 0.71049291,
 'CO,2014': 0.54463571,
 'CO,2016': 0.72087783,
 'CT,2008': 0.66642547,
 'CT,2010': 0.45959899,
 'CT,2012': 0.60998958,
 'CT,2014': 0.42533553,
 'CT,2016': 0.65427256,
 'DC,2008': 0.6169486,
 'DC,2010': 0.29573914,
 'DC,2012': 0.61965692,
 'DC,2014': 0.35791108,
 'DC,2016': 0.61113918,
 'DE,2008': 0.65832853,
 'DE,2010

In [837]:
vep_turnout_list = []
for index, row in result_df.iterrows():
    vep_turnout_list.append(state_year_turnout_dict[str(row.state_abbv) + "," + str(row.year)])

In [838]:
vep_turnout_list

[0.60952842,
 0.43060836,
 0.58890975,
 0.33212399,
 0.59257072,
 0.68277264,
 0.52556038,
 0.59180635,
 0.54381794,
 0.61800861,
 0.34115994,
 0.56217176,
 0.57360554,
 0.41521883,
 0.53292167,
 0.52904886,
 0.37541923,
 0.50954992,
 0.40142861,
 0.53103119,
 0.61735171,
 0.45050842,
 0.55893815,
 0.30784670000000003,
 0.5840137,
 0.71601105,
 0.50696188,
 0.71049291,
 0.54463571,
 0.72087783,
 0.66642547,
 0.45959899,
 0.60998958,
 0.42533553,
 0.65427256,
 0.65832853,
 0.47842926,
 0.62660235,
 0.34355411,
 0.64607763,
 0.6169486,
 0.29573914,
 0.61965692,
 0.35791108,
 0.61113918,
 0.66630977,
 0.42283514,
 0.63962775,
 0.43262726,
 0.65744931,
 0.40592468,
 0.58650154,
 0.38530818,
 0.59887046,
 0.62731379,
 0.49035606,
 0.4009361,
 0.444745,
 0.36494294,
 0.43036032,
 0.64843172,
 0.42772385,
 0.60876757,
 0.39604637,
 0.60876042,
 0.64256495,
 0.43194485,
 0.59303057,
 0.40949884,
 0.63357818,
 0.6030463,
 0.38006222,
 0.55963689,
 0.28824309,
 0.57858533,
 0.69656885,
 0.507902

In [839]:
print(len(result_df))
print(len(vep_turnout_list))

255
255


In [840]:
result_df['vep_turnout'] = vep_turnout_list

In [842]:
result_df = result_df.drop(columns=['state_abbv'])

In [843]:
result_df.head()

,state_fips,age_group,year,data,vep_turnout
0,Alabama,Ages 65 and over,2008,636944,0.609528
1,Alabama,Ages 65 and over,2010,660854,0.430608
2,Alabama,Ages 65 and over,2012,698055,0.588910
3,Alabama,Ages 65 and over,2014,741948,0.332124
4,Alabama,Ages 65 and over,2016,783936,0.592571


In [844]:
session.set_keyspace(keyspace)
query = "CREATE TABLE Table9 (key float primary key, state_fips text, year int, age_group text, data int, vep_turnout float)"
session.execute(query)

In [845]:
session.set_keyspace(keyspace)
query = "INSERT INTO Table9 (key, state_fips, year, age_group, data, vep_turnout) values (?, ?, ?, ?, ?, ?)"
prepared = session.prepare(query)
for index, row in result_df.iterrows():
    parameter_list = []
    parameter_list.append(index)
    parameter_list.append(row['state_fips'])
    parameter_list.append(row['year'])
    parameter_list.append(row['age_group'])
    parameter_list.append(row['data'])
    parameter_list.append(row['vep_turnout'])
    session.execute(prepared, tuple(parameter_list))

In [846]:
## Table10

In [847]:
input_path = "./Dataset/epi_indicators-all_years.csv"
updated_data_path = "./Dataset/updated_epi_indicators.csv"
cluster_ip = '54.164.78.8'
keyspace = 'election'

table_name = 'Table10'
column_list = ['state_abbv', 'vep_turnout', 'uocava_rej', 'year']

result_dict = dict()

preprocessing(input_path, updated_data_path)

result_df = get_column_df(updated_data_path, column_list)

In [848]:
state_name = []
for index, row in result_df.iterrows():
    state_name.append(abbv_name_dic[row['state_abbv']])

result_df['state_fips'] = state_name

In [849]:
result_df = result_df.drop(columns=['state_abbv'])

In [850]:
result_df.head()

,vep_turnout,uocava_rej,year,state_fips
0,0.682773,0.042965,2008,Alaska
1,0.525560,0.042495,2010,Alaska
2,0.591806,0.082484,2012,Alaska
3,0.543818,0.050325,2014,Alaska
4,0.618009,0.079330,2016,Alaska


In [851]:
session.set_keyspace(keyspace)
query = "CREATE TABLE Table10 (key float primary key, state_fips text, year int, uocava_rej float, vep_turnout float)"
session.execute(query)

In [852]:
session.set_keyspace(keyspace)
query = "INSERT INTO Table10 (key, state_fips, year, uocava_rej, vep_turnout) values (?, ?, ?, ?, ?)"
prepared = session.prepare(query)
for index, row in result_df.iterrows():
    parameter_list = []
    parameter_list.append(index)
    parameter_list.append(row['state_fips'])
    parameter_list.append(row['year'])
    parameter_list.append(row['uocava_rej'])
    parameter_list.append(row['vep_turnout'])
    session.execute(prepared, tuple(parameter_list))